In [1]:
import cv2
import os
import csv
import numpy as np
import mediapipe as mp
from matplotlib import pyplot as plt
import time

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [3]:
def mediapipe_detection(frame, model):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
   # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [10]:
def record_landmarks(results, action):
    try:
        pose = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

        right_hand = results.right_hand_landmarks.landmark
        right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())

        left_hand = results.left_hand_landmarks.landmark
        left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand]).flatten())
 
        row = pose_row + right_hand_row + left_hand_row
        row.insert(0, action)

        with open('coords.csv', mode='a', newline='') as f:
            csv_writer_obj = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer_obj.writerow(row)
            
    except:
        pass
        

# Testing Camera and recording

In [13]:
#TESTING CAMERA

cam = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.5) as holistic:
    while cam.isOpened():
        ret, frame = cam.read()
    
        image, results = mediapipe_detection(frame, holistic)
        
        draw_landmarks(image, results)
        
        cv2.imshow("Sign Language Converter", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        #print(results.right_hand_landmarks) 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()

In [26]:
cam.release()
cv2.destroyAllWindows()

In [11]:
#CREATE HEADER IN CSV
NUM_COORDS = 75
#num_coords = len(results.pose_landmarks.landmark) + len(results.right_hand_landmarks.landmark) + len(results.left_hand_landmarks.landmark)
header_list = ["action"]
for i in range(1, NUM_COORDS+1):
    header_list += ["x{}".format(i),"y{}".format(i),"z{}".format(i),"v{}".format(i)]

with open('coords.csv', mode='w', newline='') as f:
    csv_writer_obj = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer_obj.writerow(header_list)

In [15]:
DATA_PATH = os.path.join('MP_data')
actions = np.array(["Ambulance", "Doctor", "Hurt"])
no_sequences = 50
sequence_length = 30

In [16]:
#recording per sequence

cam = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.5) as holistic:
    
    for action in actions:
        for sequence in range(no_sequences):
            for frame_no in range(sequence_length):
                
                ret, frame = cam.read()
                image, results = mediapipe_detection(frame, holistic)
                draw_landmarks(image, results)
                
                #print(results.right_hand_landmarks)
                #extract_keypoints(results)
                if frame_no == 0:
                    cv2.putText(image, "Collecting Video for {} Video number {}".format(action, sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                     cv2.putText(image, "Collecting Video for {} Video number {}".format(action, sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)

                record_landmarks(results, action)
                #npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_no))
                #np.save(npy_path, keypoints)
                
                cv2.imshow("Input Sign Language", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    cam.release()
                    cv2.destroyAllWindows()

    cam.release()
    cv2.destroyAllWindows()

In [60]:
#RECORDING WITHOUT ANY SEQUENCE

cam = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.5) as holistic:
    while cam.isOpened():
        ret, frame = cam.read()
    
        image, results = mediapipe_detection(frame, holistic)
        
        draw_landmarks(image, results)

        record_landmarks(results)
        
        cv2.imshow("Sign Language Converter", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        #print(results.right_hand_landmarks) 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()

# SPLIT DATA

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [18]:
df = pd.read_csv("coords.csv")

In [19]:
X = df.drop('action', axis=1) #feature
y = df['action'] #target 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [20]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import accuracy_score
import pickle


In [21]:
#TRAINING

pipelines = {
    'lr' : make_pipeline(StandardScaler(), LogisticRegression()),
    'rc' : make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf' : make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb' : make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [28]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [30]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.9987146529562982
rc 0.9987146529562982
rf 0.9974293059125964
gb 0.9987146529562982


In [31]:
fit_models['rf'].predict(X_test)

array(['Ambulance', 'Doctor', 'Doctor', 'Hurt', 'Doctor', 'Hurt',
       'Ambulance', 'Doctor', 'Doctor', 'Hurt', 'Ambulance', 'Hurt',
       'Ambulance', 'Hurt', 'Doctor', 'Ambulance', 'Ambulance',
       'Ambulance', 'Doctor', 'Ambulance', 'Hurt', 'Ambulance', 'Hurt',
       'Hurt', 'Ambulance', 'Doctor', 'Doctor', 'Hurt', 'Doctor',
       'Ambulance', 'Hurt', 'Ambulance', 'Ambulance', 'Hurt', 'Doctor',
       'Ambulance', 'Doctor', 'Doctor', 'Ambulance', 'Hurt', 'Doctor',
       'Doctor', 'Ambulance', 'Doctor', 'Hurt', 'Ambulance', 'Doctor',
       'Hurt', 'Doctor', 'Hurt', 'Ambulance', 'Hurt', 'Hurt', 'Doctor',
       'Doctor', 'Ambulance', 'Doctor', 'Ambulance', 'Doctor', 'Hurt',
       'Hurt', 'Doctor', 'Hurt', 'Hurt', 'Ambulance', 'Hurt', 'Ambulance',
       'Ambulance', 'Doctor', 'Ambulance', 'Ambulance', 'Ambulance',
       'Ambulance', 'Hurt', 'Hurt', 'Ambulance', 'Doctor', 'Ambulance',
       'Doctor', 'Ambulance', 'Ambulance', 'Doctor', 'Doctor', 'Hurt',
       'Hurt', 'Doc

In [32]:
y_test

235     Ambulance
914        Doctor
1290       Doctor
2238         Hurt
1140       Doctor
          ...    
1926         Hurt
448     Ambulance
1047       Doctor
961        Doctor
971        Doctor
Name: action, Length: 778, dtype: object

In [33]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'], f)

In [34]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [35]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

# Real Time Testing

In [38]:
cam = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.5) as holistic:
    while cam.isOpened():
        ret, frame = cam.read()
    
        image, results = mediapipe_detection(frame, holistic)
        draw_landmarks(image, results)
        
        #print(results.right_hand_landmarks)

        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
    
            right_hand = results.right_hand_landmarks.landmark
            right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())
    
            left_hand = results.left_hand_landmarks.landmark
            left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand]).flatten())
     
            row = pose_row + right_hand_row + left_hand_row
            
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)

            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        except:
            pass


        cv2.imshow("Sign Language Converter", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()


C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


Doctor [4.42322776e-02 9.55767722e-01 7.47017071e-28]
Doctor [5.07528357e-03 9.94924716e-01 5.71921159e-28]


C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


Doctor [4.71647458e-03 9.95283525e-01 5.66086652e-27]
Doctor [1.87509774e-03 9.98124902e-01 4.17922878e-26]


C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


...

Hurt [3.87995710e-05 4.21437224e-01 5.78523977e-01]
Doctor [1.07605389e-05 6.76943024e-01 3.23046216e-01]


C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


Doctor [4.09227283e-06 7.72426183e-01 2.27569725e-01]
Hurt [1.71085920e-05 4.83571867e-01 5.16411025e-01]


C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


Doctor [4.97095575e-06 5.93571594e-01 4.06423435e-01]
